In [ ]:
from pythonnet import load
load("coreclr")

import os
import clr
clr.AddReference(os.path.abspath("../src/DerivaSharp/bin/Release/net9.0/win-x64/publish/DerivaSharp.dll"))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from DerivaSharp.Instruments import EuropeanOption, OptionType
from DerivaSharp.PricingEngines import AnalyticEuropeanEngine, PricingContext
from System import DateOnly

In [ ]:
type = OptionType.Call
strike = 100.0
startDate = DateOnly(2025, 1, 6)
endDate = startDate.AddDays(90)
vol = 0.3
r = 0.04
q = 0.01

option = EuropeanOption(type, strike, startDate, endDate)
engine = AnalyticEuropeanEngine()

In [ ]:
metrics = {
    "value": engine.Value,
    "delta": engine.Delta,
    "gamma": engine.Gamma,
    "speed": engine.Speed,
    "theta": engine.Theta,
    "charm": engine.Charm,
    "color": engine.Color,
    "vega": engine.Vega,
    "vanna": engine.Vanna,
    "zomma": engine.Zomma,
    "rho": engine.Rho,
}

data = []
spots = np.arange(50, 150, 1, dtype=np.float64)
for spot in spots:
    context = PricingContext(spot, startDate, vol, r, q)
    row = {name: func(option, context) for name, func in metrics.items()}
    row["spot"] = spot
    data.append(row)
df = pd.DataFrame(data)

In [ ]:
greek_columns = [col for col in df.columns if col != "spot"]
fig, axes = plt.subplots(len(greek_columns), 1, figsize=(4, 3 * len(greek_columns)))
for ax, col in zip(axes, greek_columns):
    ax.plot(df["spot"], df[col], label=col, color="C0")
    ax.set_title(col.capitalize())
    ax.grid(True)
plt.tight_layout()
plt.show()